In [1]:
# from urllib.request import urlretrieve

# def load_data(download=True):
#     if download:
#         data_path, _ = urlretrieve(
#             "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
#             "./adult.csv"
#         )
#         print("adult dataset download")


# load_data()

downloaded


<!-- https://blog.csdn.net/Power1_Power2/article/details/79664830
https://blog.csdn.net/Power1_Power2/article/details/81559344
https://blog.csdn.net/Power1_Power2/article/details/81838532 -->


<!-- https://blog.csdn.net/Power1_Power2/article/details/79664830 -->
<!-- https://blog.csdn.net/Power1_Power2/article/details/81559344 -->
<!-- https://blog.csdn.net/Power1_Power2/article/details/81838532 -->

1. 连续性数值特征，如age,最好处理
2. 二分类文本信息，二分法处理
3. 多类别文本信息

- age:连续性数值变量；可能的处理方法：分年龄段；
- workcass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov,State-gov, Without-pay, Never-worked.
 ：雇主类型，多类别，一般处理方法：化为数值类别，比如以上八个可以分别表示为1-8（仅为示例，本文并不推荐）；
- fnlwgt: 连续性数值变量；人口普查员认为观察值的人数。该变量在本文不被使用，笔者认为该特征并不重要。
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm,Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th,Preschool.受教育程度，多类别数据，处理方法同workcass；
- education-num: 连续性数值变量，受教育水平年限，一般来讲，该值越大，工资越高；
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated,Widowed, Married-spouse-absent, Married-AF-spouse.婚姻状况，多类别数据；
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial,Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical,Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv,Armed-Forces.职业，多类别数据；
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative,Unmarried.群体性关系，多类别数据；
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.种族，多类别数据，虽然美国反对种族歧视，但是实际上这个在区分美国工资的时候蛮重要；
- sex: Female, Male.性别，最简单的二分法（0&1）；
- capital-gain: 资本收益，连续数值；
- capital-loss: 资本损失，连续数值；
- hours-per-week: 工作时间，连续数值；
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada,Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China,Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico,Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti,Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia,El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.国籍，多类别数据；
- result：结果：“>50K”或“<=50K”，二分类数据，也是本文机器学习的目的（0&1）；

In [173]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [142]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "result"]

# assign column names
adult = pd.read_csv(
  "./adult.csv",
  names=col_names,
)
# print(adult[:10])

In [135]:
# check null value
# adult.info()

# check null value
# adult.isnull().any()

In [136]:
# adult.shape

In [143]:
adult_clean = adult.replace(regex=[r"\?|\.|\$"], value=np.nan)

In [138]:
adult_clean.isnull().any()

# For some data sets, some fill in the mean with null values, but this data set needs to predict income.
# So I directly remove it here without affecting the result. If the mean value is filled here, the training set will be affected by the accuracy.

age               False
workclass          True
fnlwgt            False
education         False
education-num     False
marital-status    False
occupation         True
relationship      False
race              False
sex               False
capital-gain      False
capital-loss      False
hours-per-week    False
native-country     True
result            False
dtype: bool

In [144]:
adult = adult_clean.dropna(how="any")

# adult.shape

In [160]:
# remove features
adult_final = adult.drop(["fnlwgt"], axis=1)

adult_final_xgboot = adult.drop(["fnlwgt"], axis=1)
adult_final_xgboot["result"] = adult_final_xgboot["result"].apply(
    lambda x: 0 if x == "<=50K" else 1
)

In [147]:
col_names = ["age", "workclass", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "result"]

In [162]:
# adult_final.head()
# adult_final_xgboot.head()

In [169]:
# 75% / 25%
X_train, X_test, y_train, y_test=train_test_split(
  adult_final[col_names[1:13]],
  adult_final[col_names[13]],
  test_size=0.25,
  random_state=33,
)

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    adult_final_xgboot[col_names[1:13]],
    adult_final_xgboot[col_names[13]],
    test_size=0.25,
    random_state=33,
)

In [165]:
print(X_train.shape)
print(X_test.shape)

print(X_train_xgb.shape)
print(X_test_xgb.shape)

(22621, 12)
(7541, 12)
(22621, 12)
(7541, 12)


### Feature engineering

- One-Hot Encoding: Use when each data point belongs to one and only one category (e.g., types of animals, types of fruits).

```
'cat' -> [1, 0, 0]
'dog' -> [0, 1, 0]
'bird' -> [0, 0, 1]
```

- Multi-Hot Encoding: Use when each data point can belong to multiple categories simultaneously (e.g., tags for a blog post, genres of a movie).

```
'cat' and 'bird' -> [1, 0, 1]
```

In [166]:
dict_vect = DictVectorizer(sparse=False)

## random forest

In [171]:
X_train_formatted = dict_vect.fit_transform(
  X_train.to_dict(orient="records")
)
# dict_vect.feature_names_

In [172]:
X_test_formatted = dict_vect.transform(
  X_test.to_dict(orient="records")
)

In [83]:
rfc = RandomForestClassifier()

In [175]:
# cross-validation score
cross_val_score(
  rfc,
  X_train_formatted,
  y_train,
  cv=5
).mean()

0.8394413294840044

In [177]:
rfc.fit(X_train_formatted, y_train)
rfc_y_predict = rfc.predict(X_test_formatted)

rfc.score(X_test_formatted, y_test)

0.8427264288555895

## XGBoost

In [178]:
xgbc = XGBClassifier()

In [179]:
X_train_xgb_formatted = dict_vect.fit_transform(
  X_train_xgb.to_dict(orient="records")
)

X_test_XGB_formatted = dict_vect.transform(
  X_test_xgb.to_dict(orient="records")
)

In [186]:
# X_train_xgb_formatted.shape
# X_test_XGB_formatted.shape
# y_train_xgb
# y_test_xgb

In [187]:
cross_val_score(
  xgbc,
  X_train_xgb_formatted,
  y_train_xgb,
  cv=5
).mean()

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jiaronghe/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jiaronghe/.pyenv/versions/3.11.3/lib/python3.11/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/Users/jiaronghe/.pyenv/versions/3.11.3/lib/python3.11/site-packages/xgboost/sklearn.py", line 1471, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [1]


In [130]:
# xgbc.fit(X_train, y_train)
# xgbc_y_predict = xgbc.predict(X_test)
# xgbc.score(X_test, y_test)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [1]

In [90]:
print(rfc_y_predict[1:20])
print(y_test[1:20])

[' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K'
 ' >50K' ' <=50K' ' >50K' ' <=50K' ' <=50K' ' >50K' ' >50K' ' <=50K'
 ' <=50K' ' <=50K' ' >50K']
16568     <=50K
16406     <=50K
13051     <=50K
29453     <=50K
31063     <=50K
6741      <=50K
29557     <=50K
32097     <=50K
19604     <=50K
30108     <=50K
20128     <=50K
13205     <=50K
2493      <=50K
1193      <=50K
31161      >50K
10252     <=50K
20591     <=50K
8290       >50K
25915      >50K
Name: result, dtype: object


In [94]:
from sklearn.metrics import classification_report
print('Random forest accuracy:')
print(
  classification_report(
    y_test,
    rfc_y_predict,
    target_names=['<=50K', '>50K']
  )
)


# print('XGBoost accuracy:')
# print(classification_report(y_test,xgbc_y_predict,target_names=['result']))

Random forest accuracy:
              precision    recall  f1-score   support

       <=50K       0.87      0.92      0.90      5613
        >50K       0.73      0.60      0.66      1928

    accuracy                           0.84      7541
   macro avg       0.80      0.76      0.78      7541
weighted avg       0.83      0.84      0.84      7541

